In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import nltk

import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


In [ ]:
#### Removing Nulls

# Before Dropping any columns or rows

# train_data = train_data.replace(to_replace=np.nan,value=-1) ## I dont know why it was not showing null values in dataframe so i converted it to -1 for sometime.

# def replace_title_test_nan(df):
#   for i in range(len(df.TITLE)):
#     if df.TITLE[i] == -1:
#       df.TITLE[i] = df.BULLET_POINTS[i]
#   return df

# df = df.replace(to_replace=-1,value=np.nan)




## **Train Data**

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/dataset/nulls_removed.csv")
train_df.head()

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5


In [ ]:
df = train_df.iloc[:20000,[0,3]]

In [ ]:
df.head()

,TITLE,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch",0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),3
4,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,5


## Test Data

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/dataset/nulls_removed_test2.csv")
test_df.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [ ]:
test_df.drop(["BULLET_POINTS","PRODUCT_ID","BRAND"],axis=1,inplace=True)
test_df.head()

,TITLE,DESCRIPTION
0,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK
1,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...
2,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...
3,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...
4,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat..."


In [ ]:
norm_test_data = pre_process_corpus(test_df["TITLE"])


100%|██████████| 103989/103989 [00:14<00:00, 7427.65it/s]


In [ ]:
norm_test_data

['command 3m small kitchen hooks white decorate damage free easy on easy off 9 hooks 12 strips multipack 17067vp',
 'oneal jump hardware jag unisexadult glove blackred 11',
 'nfl detroit lions portable party fridge 158 quart',
 'panasonic single line kxts880mx corded phone white',
 'zero baby girls 100 cotton innerwear bloomer drawer panties 03 months multicolour pack of 6',
 'nizhi tt028 mp3 mini digital portable music player micro sd usb fm radio black',
 'nba los angeles clippers dorm room fridge 17 cubic feet',
 'one industries zero zerope gloves cyanred medium',
 '3xlarge shaf international deer skin fingerless gloves with gel padded palm black xxxlarge',
 'makita d36049',
 'gioiabazar leather flip designer stripe wallet case cover pouch for motorola moto g 2nd gen 2014 khaki',
 'axo fight gloves blackfluorescentyellow size 7xsmall',
 'vpsk silver ceramic aluminium pot heavy patilarice boiler tope no 18 550l',
 'hjc helmets mens cold weather gloves blackred xxxlarge',
 'dog waterp

In [ ]:
%%time

# transform test reviews into features
cv_test_features = cv.transform(norm_test_data)
tv_test_features = tv.transform(norm_test_data)

CPU times: user 6.38 s, sys: 84.1 ms, total: 6.46 s
Wall time: 6.47 s


In [ ]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

BOW model:> Train features shape: (16000, 7884)  Test features shape: (103989, 7884)
TFIDF model:> Train features shape: (16000, 7884)  Test features shape: (103989, 7884)


In [ ]:
preds = lr.predict(cv_test_features)

## Submission

In [ ]:
final = pd.read_csv("/content/drive/MyDrive/dataset/nulls_removed_test2.csv")

In [ ]:
final.drop(["BULLET_POINTS","TITLE"],axis=1,inplace=True)

In [ ]:
final.head()

,PRODUCT_ID,BRAND
0,1,Command
1,2,O'Neal
2,3,Boelter Brands
3,4,Panasonic
4,5,Zero


In [ ]:
final["BRAND"] = preds

In [ ]:
final.head()

,PRODUCT_ID,BRAND
0,1,4
1,2,55
2,3,4
3,4,11
4,5,1922


In [ ]:
final.rename(columns={'BRAND': 'BROWSE_NODE_ID'}, inplace=True)

In [ ]:
final.head()

,PRODUCT_ID,BROWSE_NODE_ID
0,1,4
1,2,55
2,3,4
3,4,11
4,5,1922


In [ ]:
final.to_csv("1lr.csv",index=False)

In [ ]:
che = pd.read_csv("/content/1lr.csv")
che.head()

,PRODUCT_ID,BROWSE_NODE_ID
0,1,4
1,2,55
2,3,4
3,4,11
4,5,1922


## Text Wrangling & Normalization


In [ ]:
def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [ ]:
norm_data = pre_process_corpus(df["TITLE"])

100%|██████████| 20000/20000 [00:02<00:00, 7811.15it/s]


In [ ]:
norm_data

['pete the cat bedtime blues doll 145 inch',
 'the new yorker nyhm014 refrigerator magnet 2 x 35',
 'the ultimate selfsufficiency handbook a complete guide to baking crafts gardening preserving your harvest raising animals and more',
 'amway nutrilite kids chewable iron tablets 100',
 'mens full sleeve raglan tshirts denim tshirt m size only',
 'glance womens wallet black lw21',
 'wild animals hungry brain educational flash cards for enhancing early learning of your kids3 month to 6 year',
 'hindi barakhadi educational flash cards for kids enhance early learning of your kids 3 months to 6 years',
 'white braun kf400wh aromaster 10cup coffeemaker white',
 'yamaha pcmskit portable electronic keyboard adapter with pc joystick to midi cable',
 'microsoft sidewinder gamepad pro',
 'jiffy steamer with plastic steam head beige 120v',
 'oxo good grips bpafree turkey baster with cleaning brush',
 'sony vaio roomlink network media receiver pcnamr10',
 'westcott 5684 5 x 6 feet collapsible backdr

In [ ]:
X = norm_data
y = df["BROWSE_NODE_ID"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Traditional Supervised Machine Learning Models
Feature Engineering

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:

%%time

# build BOW features on train reviews
cv = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
cv_train_features = cv.fit_transform(X_train)


# build TFIDF features on train reviews
# tv = TfidfVectorizer(use_idf=True, min_df=5, max_df=1.0, ngram_range=(1,2),
#                      sublinear_tf=True)
# tv_train_features = tv.fit_transform(X_train)

CPU times: user 2.25 s, sys: 51.7 ms, total: 2.3 s
Wall time: 2.32 s


In [ ]:

%%time

# transform test reviews into features
cv_test_features = cv.transform(X_test)
# tv_test_features = tv.transform(X_test)

CPU times: user 226 ms, sys: 623 µs, total: 227 ms
Wall time: 230 ms


In [ ]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)
# print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

BOW model:> Train features shape: (16000, 7884)  Test features shape: (4000, 7884)
TFIDF model:> Train features shape: (16000, 7884)  Test features shape: (4000, 7884)


In [ ]:

%%time

# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=1000, C=1, solver='lbfgs', random_state=42)

# train model
lr.fit(cv_train_features, y_train)

# predict on test data
lr_bow_predictions = lr.predict(cv_test_features)

CPU times: user 18min 48s, sys: 1min 33s, total: 20min 22s
Wall time: 17min 33s


In [ ]:

from sklearn.metrics import confusion_matrix, classification_report


print(classification_report(y_test, lr_bow_predictions))
pd.DataFrame(confusion_matrix(y_test, lr_bow_predictions))

              precision    recall  f1-score   support

           0       0.29      0.67      0.40         3
           1       1.00      0.67      0.80         3
           3       0.00      0.00      0.00         1
           4       0.01      0.15      0.02        13
           5       0.65      0.88      0.75        92
           6       0.83      1.00      0.91         5
           7       1.00      0.50      0.67         2
           9       0.00      0.00      0.00         1
          11       0.64      0.96      0.77        28
          16       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         2
          20       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         1
          22       0.59      0.89      0.71        18
          23       1.00      0.33      0.50         3
          31       0.00      0.00      0.00         1
          34       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1434,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1435,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1436,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
